In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_1 = pd.read_csv(folderpath + 'data_op_01.csv')
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,0.02,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,0.02,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,0.02,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,0.02,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,0.02,1,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_1 = tf_remove_sensor_15.fit_transform(data_op_1)
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,1,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_1)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
2,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1
11,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1
23,1359.28,1131.95,5.48,2222.88,183.36,2387.94,2
38,1370.11,1126.43,5.48,2222.88,183.45,2387.95,2
40,1359.77,1129.48,5.48,2222.89,182.97,2387.98,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[ 2.18753538e-01, -1.28328598e+00, -1.00000000e+00,
        -5.61707703e-01,  6.68924330e-01, -5.45027910e-01],
       [-1.13058311e+00, -1.71913583e-01, -1.00000000e+00,
        -4.90969556e-01,  1.06520396e+00, -5.45027910e-01],
       [-1.35340934e+00, -1.14172757e-03, -1.00000000e+00,
        -4.90969556e-01,  9.33110750e-01, -5.78004106e-01],
       ...,
       [ 8.83695318e-01,  9.06930839e-01, -1.00000000e+00,
        -1.12761287e+00, -1.10112469e+00, -1.40240900e+00],
       [ 1.05523615e+00,  9.55722798e-01, -1.00000000e+00,
        -1.26908916e+00, -1.60307888e+00, -1.13859943e+00],
       [ 2.31968661e+00,  1.83397806e+00, -1.00000000e+00,
        -1.69351804e+00, -2.58056864e+00, -1.83109955e+00]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 2, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)
# SVM
svm_clf = SVC()

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf, 'SVM': svm_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))},
                   'SVM': {'gamma': [2*i for i in range(-15,3)], 'C': [2*i for i in range(-5,15)],
                           'kernel': ['poly', 'rbf', 'sigmoid']}
                  }

In [13]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=20, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=349 ..........................................
[CV] hidden_layer_sizes=322 ..........................................
[CV] hidden_layer_sizes=322 ..........................................
[CV] hidden_layer_sizes=322 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  42 out of 200 | elapsed:  6.5min remaining: 24.4min


[CV] . hidden_layer_sizes=380, score=0.6384839650145773, total= 6.5min
[CV] hidden_layer_sizes=428 ..........................................
[CV] . hidden_layer_sizes=876, score=0.6370262390670554, total= 6.3min
[CV] hidden_layer_sizes=428 ..........................................
[CV] . hidden_layer_sizes=380, score=0.6067251461988304, total= 6.5min
[CV] hidden_layer_sizes=428 ..........................................
[CV] .. hidden_layer_sizes=66, score=0.6317321688500728, total= 2.5min
[CV] hidden_layer_sizes=428 ..........................................
[CV] . hidden_layer_sizes=542, score=0.6370262390670554, total= 6.7min
[CV] hidden_layer_sizes=428 ..........................................
[CV] . hidden_layer_sizes=328, score=0.6545189504373178, total= 6.8min
[CV] hidden_layer_sizes=428 ..........................................
[CV] . hidden_layer_sizes=328, score=0.6399416909620991, total= 6.9min
[CV] hidden_layer_sizes=428 ..........................................
[CV] .

[Parallel(n_jobs=100)]: Done  83 out of 200 | elapsed:  8.8min remaining: 12.5min


[CV] hidden_layer_sizes=866 ..........................................
[CV] .. hidden_layer_sizes=66, score=0.6501457725947521, total= 3.8min
[CV] . hidden_layer_sizes=374, score=0.6355685131195336, total= 8.7min
[CV] hidden_layer_sizes=866 ..........................................
[CV] hidden_layer_sizes=866 ..........................................
[CV] . hidden_layer_sizes=374, score=0.6273653566229985, total= 8.8min
[CV] hidden_layer_sizes=866 ..........................................
[CV] . hidden_layer_sizes=876, score=0.6394160583941606, total= 8.8min
[CV] hidden_layer_sizes=866 ..........................................
[CV] . hidden_layer_sizes=542, score=0.6559766763848397, total= 9.0min
[CV] hidden_layer_sizes=866 ..........................................
[CV] . hidden_layer_sizes=658, score=0.6364963503649635, total= 9.0min
[CV] hidden_layer_sizes=866 ..........................................
[CV] . hidden_layer_sizes=542, score=0.6064139941690962, total= 9.3min
[CV] h

...

[Parallel(n_jobs=100)]: Done 165 out of 200 | elapsed: 14.0min remaining:  3.0min


[CV] . hidden_layer_sizes=282, score=0.6302765647743813, total= 6.4min
[CV] . hidden_layer_sizes=851, score=0.6166180758017493, total=10.6min
[CV] . hidden_layer_sizes=187, score=0.6516034985422741, total= 5.7min
[CV] . hidden_layer_sizes=428, score=0.6154970760233918, total= 7.0min
[CV] . hidden_layer_sizes=851, score=0.6428571428571429, total=10.2min
[CV] . hidden_layer_sizes=282, score=0.6209912536443148, total= 6.2min
[CV] . hidden_layer_sizes=876, score=0.6084425036390102, total=14.1min
[CV] . hidden_layer_sizes=866, score=0.6394160583941606, total= 5.0min
[CV] .. hidden_layer_sizes=915, score=0.619533527696793, total= 8.9min
[CV] . hidden_layer_sizes=468, score=0.6273653566229985, total= 6.9min
[CV] . hidden_layer_sizes=866, score=0.6302765647743813, total= 5.7min
[CV] . hidden_layer_sizes=851, score=0.6341107871720116, total=10.9min
[CV] . hidden_layer_sizes=915, score=0.6122448979591837, total= 9.2min
[CV] .. hidden_layer_sizes=851, score=0.604075691411936, total=11.8min
[CV] .

[Parallel(n_jobs=100)]: Done 200 out of 200 | elapsed: 15.2min finished


Elapsed time:
917.928439617157
Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-12, C=16 .....................................
[CV] kernel=rbf, gamma=-28, C=12 .....................................
[CV] kernel=rbf, gamma=-28, C=12 .....................................
[CV] kernel=rbf, gamma=-28, C=12 .....................

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/navar/anaconda3/envs/phm08ds/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/navar/anaco

## Savel results, models and pipeline to a .pkl file 

In [ ]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [ ]:
experiment.save_models(random_search)
experiment.save_pipeline(data_preprocessing)